# Deploy Model to Run on Region of Interest
Note: Requires Descartes Labs access

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import sys

import descarteslabs as dl
import geopandas as gpd
from tensorflow.keras.models import load_model
from tensorflow import keras
from tqdm.notebook import tqdm

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import deploy_nn_v1

In [6]:
# User inputs
roi = 'test_region'
roi_file = f'../data/boundaries/{roi}.geojson'

patch_model_name = '44px_v2.8_2021-11-11'
patch_model_version = '44px_v2.8'
patch_model_file = '../models/' + patch_model_name + '.h5'
patch_model = load_model(patch_model_file, custom_objects={'LeakyReLU': keras.layers.LeakyReLU,
                                                           'ELU': keras.layers.ELU,
                                                           'ReLU': keras.layers.ReLU})
patch_stride = 14
patch_input_shape = patch_model.input_shape[1]

# Note on dates: The date range should be longer than the spectrogram length.
# Starting on successive mosaic periods (typically: monthly), as many
# spectrograms are created as fit in the date range.
start_date = '2020-01-01'
end_date = '2021-02-01'

mosaic_period = 4
mosaic_method = 'median'

patch_product_id = f'earthrise:mining_{roi}_v{patch_model_version}_{start_date}_{end_date}_period_{mosaic_period}_method_{mosaic_method}' 
product_name = patch_product_id.split(':')[-1]  # Arbitrary string - optionally set this to something more human readable.

run_local = False # If False, the model prediction tasks are async queued and sent to DL for processing.

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2021-11-17 22:12:09.943571: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-17 22:12:09.943763: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# If running locally, get results faster by setting smalle tilesize (100?)
# If running on Descartes, use tilesize 900

if run_local:
    tilesize = 100
else:
    tilesize = 900

padding = patch_input_shape - patch_stride

args = [
    '--roi_file',
    roi_file,
    '--patch_product_id',
    patch_product_id,
    '--product_name',
    product_name,
    '--patch_model_name',
    patch_model_name,
    '--patch_model_file',
    patch_model_file,
    '--patch_stride',
    str(patch_stride),
    '--mosaic_period',
    str(mosaic_period),
    '--mosaic_method',
    mosaic_method,
    '--start_date',
    start_date,
    '--end_date',
    end_date,
    '--pad',
    str(padding),
    '--tilesize',
    str((tilesize // patch_input_shape) * patch_input_shape - padding)
]
if run_local:
    args.append('--run_local')

Launch Descartes job. Monitor at https://monitor.descarteslabs.com/

In [8]:
# Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
%cd ../scripts
%pwd

/Users/ckruse/Documents/earthrise/mining-detector/scripts


'/Users/ckruse/Documents/earthrise/mining-detector/scripts'

In [9]:
# Check if patch feature collection exists. If it does, delete the FC
fc_ids = [fc.id for fc in dl.vectors.FeatureCollection.list() if patch_product_id in fc.id]
if len(fc_ids) > 0:
    fc_id = fc_ids[0]
    print("Existing product found.\nDeleting", fc_id)
    dl.vectors.FeatureCollection(fc_id).delete()
else:
    print("No existing product found.\nCreating", patch_product_id)

No existing product found.
Creating earthrise:mining_test_region_v44px_v2.8_2020-01-01_2021-02-01_period_4_method_median


In [10]:
deploy_nn_v1.main(args)

Split ROI into 25 tiles
Model 44px_v2.8_2021-11-11 found in DLStorage.
Creating product earthrise:mining_test_region_v44px_v2.8_2020-01-01_2021-02-01_period_4_method_median_patches


100%|██████████| 25/25 [00:04<00:00,  5.25it/s]


# Download Data

### Download Patch Classifier Feature Collection

In [11]:
print("Downloading", patch_product_id)
fc_id = [fc.id for fc in dl.vectors.FeatureCollection.list() if patch_product_id in fc.id][0]
fc = dl.vectors.FeatureCollection(fc_id)
region = gpd.read_file(roi_file)['geometry']
    
features = []
for elem in tqdm(fc.filter(region).features()):
    features.append(elem.geojson)
results = gpd.GeoDataFrame.from_features(features)

if len(results) == 0:
    print("No results found for", product_name)
else:
    basepath = os.path.join('../data/outputs/', patch_model_version)
    print("Saving to", basepath)
    if not os.path.exists(basepath):
        os.makedirs(basepath)
    results.to_file(f"{basepath}/{product_name}.geojson", driver='GeoJSON')
    print(len(features), 'features found')

  0%|          | 0/37 [00:00<?, ?it/s]

Saving to ../data/outputs/44px_v2.8
37 features found


# Batched Run
Deploy model on a folder of boundary files rather than a single ROI

## Define parameters that are consistent across regions

In [ ]:
patch_model_name = '44px_v2.8_2021-11-11'
patch_model_version = '44px_v2.8'
patch_model_file = '../models/' + patch_model_name + '.h5'
patch_model = load_model(patch_model_file, custom_objects={'LeakyReLU': keras.layers.LeakyReLU,
                                                           'ELU': keras.layers.ELU,
                                                           'ReLU': keras.layers.ReLU})
patch_stride = 14
patch_input_shape = patch_model.input_shape[1]

# Note on dates: The date range should be longer than the spectrogram length.
# Starting on successive mosaic periods (typically: monthly), as many
# spectrograms are created as fit in the date range.
start_date = '2020-01-01'
end_date = '2021-02-01'

mosaic_period = 4
mosaic_method = 'median'

run_local = False # If False, the model prediction tasks are async queued and sent to DL for processing.

### Load folder of boundary files

In [ ]:
boundary_folder = '../data/boundaries/amazon_basin'
region_list = [f.split('.')[0] for f in os.listdir(boundary_folder)]
region_list

## Deploy model on region
This process will take some time to deploy if the regions of interest are large

In [ ]:
for roi in sorted(region_list):
    roi_file = os.path.join(boundary_folder, roi + '.geojson')
    patch_product_id = f'earthrise:mining_{roi}_v{patch_model_version}_{start_date}_{end_date}_period_{mosaic_period}_method_{mosaic_method}' 
    product_name = patch_product_id.split(':')[-1]  # Arbitrary string - optionally set this to something more human readable.
    tilesize = 900

    # Generally, leave padding at 0
    padding = patch_input_shape - patch_stride

    args = [
        '--roi_file',
        roi_file,
        '--patch_product_id',
        patch_product_id,
        '--product_name',
        product_name,
        '--patch_model_name',
        patch_model_name,
        '--patch_model_file',
        patch_model_file,
        '--patch_stride',
        str(patch_stride),
        '--mosaic_period',
        str(mosaic_period),
        '--mosaic_method',
        mosaic_method,
        '--start_date',
        start_date,
        '--end_date',
        end_date,
        '--pad',
        str(padding), 
        '--tilesize',
        str((tilesize // patch_input_shape) * patch_input_shape - padding)
    ]
    
    # Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
    %cd ../scripts
    %pwd

    # Check if patch feature collection exists. If it does, delete the FC
    fc_ids = [fc.id for fc in dl.vectors.FeatureCollection.list() if patch_product_id in fc.id]
    if len(fc_ids) > 0:
        fc_id = fc_ids[0]
        print("Existing product found.\nDeleting", fc_id)
        dl.vectors.FeatureCollection(fc_id).delete()
    print("Deploying", roi)
    deploy_nn_v1.main(args)

## Bulk Download
Download outputs after the model runs have completed. Note, the runs must be complete, as seen on [monitor.descarteslabs.com](monitor.descarteslabs.com), not just deployed, as seen in the previous cell.

In [ ]:
# Patch classifier product download
for roi in sorted(region_list):
    roi_file = f'../data/boundaries/amazon_basin/{roi}.geojson'
    patch_product_id = f'earthrise:mining_{roi}_v{patch_model_version}_{start_date}_{end_date}_period_{mosaic_period}_method_{mosaic_method}' 
    product_name = patch_product_id.split(':')[-1]
    print("Downloading", patch_product_id)
    fc_id = [fc.id for fc in dl.vectors.FeatureCollection.list() if patch_product_id in fc.id][0]
    fc = dl.vectors.FeatureCollection(fc_id)
    region = gpd.read_file(roi_file)['geometry']

    features = []
    for elem in tqdm(fc.filter(region).features()):
        features.append(elem.geojson)
    results = gpd.GeoDataFrame.from_features(features)
    if len(results) == 0:
        print("No results found for", product_name)
    else:
        basepath = os.path.join('../data/outputs/', patch_model_version)
        print("Saving to", basepath)
        if not os.path.exists(basepath):
            os.makedirs(basepath)
        results.to_file(f"{basepath}/{product_name}.geojson", driver='GeoJSON')
        print(len(features), 'features found')